## Trabajando sobre un csv de INDEC, esto es un poco de lo que se puede hacer con la librería Pandas, con funciones como:

- [groupby()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html)
- [merge()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html)
- [pivot_table()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.pivot_table.html)
- [Excelwriter()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.ExcelWriter.html)

*P. D.* Intentaré cambiar el código para poder ver y trabajar el zip de INDEC sin bajarlo.
No tira el link para leer la URL.
Luego subiré un Excel o CSV con las descripciones de los NCM, aunque pueden bajarlo del sistema de consulta de Comercio Exterior de INDEC:

https://www.indec.gob.ar/indec/web/Nivel4-Tema-3-2-124  

-> Pestaña **'Nomencladores'**, bajando la última enmienda disponible (la más reciente/actual).

Como dato de color: el producto más exportado (mayor cantidad de transacciones) de entre los 10 más exportados en 2020, según este archivo (exponm20) bajado con datos hasta nov-2020, es lógicamente el **maíz en grano (NCM 10059010)**; mientras que lo que más U$D generó, dentro de este top 10, fue la **Harina y "pellets" de la extracción del aceite de soja (NCM 23040010)**.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np

In [4]:
exponm = pd.read_excel('/content/drive/MyDrive/Excels/exponm.xlsx')
df = pd.DataFrame(exponm)
df.head(10)

,Año,Mes,NCM,Pdes,Pnet(kg),FOB(u$s)
0,2020,1,1012100,208,NaN,s2 ...
1,2020,1,1012100,212,NaN,s2 ...
2,2020,1,1012100,225,NaN,s2 ...
3,2020,1,1012100,999,3250.0,35580
4,2020,1,1012900,208,NaN,s2 ...
5,2020,1,1012900,212,52450.0,1064400
6,2020,1,1012900,223,NaN,s3 ...
7,2020,1,1012900,225,NaN,s2 ...
8,2020,1,1012900,331,NaN,s4 ...
9,2020,1,1012900,335,NaN,s4 ...


In [5]:
ok = df.shape
print(f"there're {ok[0]} rows and {ok[1]} columns")

there're 143714 rows and 6 columns


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 143714 entries, 0 to 143713
Data columns (total 6 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   Año       143714 non-null  int64  
 1   Mes       143714 non-null  int64  
 2   NCM       143714 non-null  int64  
 3   Pdes      143714 non-null  int64  
 4   Pnet(kg)  34635 non-null   float64
 5   FOB(u$s)  143714 non-null  object 
dtypes: float64(1), int64(4), object(1)
memory usage: 6.6+ MB


<br/> Antes de seguir, un poco de teoría sobre trabajo con columnas:
*    .nunique() Devuelve cantidad de valores únicos  en la serie
*    .unique() Devuelve la lista con los valores únicos en la serie.
*    .value_counts() Devuelve una serie con los valores únicos como índice y como valores, la cantidad de repeticiones de cada índice.

In [7]:
for col in df.columns:
  print(f'La columna {col} tiene {df[col].nunique()} valores únicos')

La columna Año tiene 1 valores únicos
La columna Mes tiene 11 valores únicos
La columna NCM tiene 5272 valores únicos
La columna Pdes tiene 201 valores únicos
La columna Pnet(kg) tiene 33237 valores únicos
La columna FOB(u$s) tiene 34574 valores únicos


In [8]:
print(df['Mes'].unique())

[ 1  2  3  4  5  6  7  8  9 10 11]


In [9]:
# Un modo de ver los valores únicos de NCM (Nomenclador Común del Mercosur) y la cantidad total que hay

ncm_unicos = pd.DataFrame(df["NCM"].unique())
ncm_unicos.columns = ['Unique NCM']
print(ncm_unicos)

unicos = df["NCM"].nunique()
print(f"\nSe repiten {unicos} nomencladores, los cuales representas bienes transados")

      Unique NCM
0        1012100
1        1012900
2        1022110
3        1041011
4        1041019
...          ...
5267    91011100
5268    91129000
5269    92089000
5270    95051000
5271    95062900

[5272 rows x 1 columns]

Se repiten 5272 nomencladores, los cuales representas bienes transados


In [10]:
# Un modo más bonito con "groupby", que además me dice cuántas veces se repite cada uno

cuenta_repes = df.groupby(['NCM']).size().reset_index(name = 'Cuenta')
cuenta_repes.sort_values(by = ['Cuenta'], ascending = False, inplace = True)
print(cuenta_repes.head(10))

print(f'\nEl dataset tiene {cuenta_repes.shape[0]} filas') # mismas que vimos en la celda anterior, está OK

# 1- El comando "reset_index" me ayuda a convertir la serie en un dataframe y a darle un título a la nueva columna creada con... (sigue)
# 1- "size()" que me dice la cantidad de veces que dicho bien se comerció
# 2- "El sort_values es para un mejor panorama de los bienes más intercambiados, o exportados en este caso
# 3- La función "groupby" es más útil si se considera más de una serie/columna

           NCM  Cuenta
314   10059010     831
615   22042100     815
354   12024200     623
5270  99980500     530
1936  38220090     409
645   23040010     395
3450  73269090     380
1594  30049099     377
2190  39269090     359
361   12060090     349

El dataset tiene 5272 filas


In [11]:
# Otro modo, este repite, pero no agrupa. Para el caso NO VA

df.agg({'NCM':'unique', 'NCM':'tolist'})

,NCM
0,1012100
1,1012100
2,1012100
3,1012100
4,1012900
...,...
143709,99980500
143710,99980500
143711,99980500
143712,99980500


In [12]:
# Otro modo, más rápido y eficiente para el caso, usando "value_counts"

cuenta_repes_1 = pd.DataFrame(df['NCM'].value_counts().reset_index(name = 'Cuenta'))
cuenta_repes_1.rename(columns = {'index':'NCM'}, inplace = True)

print(cuenta_repes_1.head(10))
print('\nLos diez más comerciados totalizan', sum(cuenta_repes_1['Cuenta'][0:10]), 'filas\n')

        NCM  Cuenta
0  10059010     831
1  22042100     815
2  12024200     623
3  99980500     530
4  38220090     409
5  23040010     395
6  73269090     380
7  30049099     377
8  39269090     359
9  12060090     349

Los diez más comerciados totalizan 5068 filas



In [13]:
# Valores U$S FOB y Kg. para los 10 bienes más exportados

top_10_NCMs = cuenta_repes_1[0:10]
print(top_10_NCMs)
print('\n', type(top_10_NCMs), "\n")
print("Chequeo: los 10 NCMs más repetidos totalizan", top_10_NCMs['Cuenta'].sum(), 'filas\n')

info_top_10 = pd.merge(df, top_10_NCMs, how = 'inner', on = 'NCM').dropna() # "dropna" para borrar filas con missing values

table = pd.pivot_table(info_top_10, index = ['Año', 'Mes'], columns = ['NCM'], 
                       values = ['Pnet(kg)', 'FOB(u$s)'], 
                       aggfunc = {'Pnet(kg)':np.sum, 'FOB(u$s)':np.sum}, 
                       margins = True, margins_name = 'Suma')
table

        NCM  Cuenta
0  10059010     831
1  22042100     815
2  12024200     623
3  99980500     530
4  38220090     409
5  23040010     395
6  73269090     380
7  30049099     377
8  39269090     359
9  12060090     349

 <class 'pandas.core.frame.DataFrame'> 

Chequeo: los 10 NCMs más repetidos totalizan 5068 filas



FOB(u$s)                ...   Pnet(kg)              
NCM           10059010      12024200  ...   99980500          Suma
Año  Mes                              ...                         
2020 1    3.143997e+08  6.060587e+07  ...   43900.42  3.580878e+09
     2    2.819127e+08  6.230973e+07  ...   31387.37  3.278241e+09
     3    6.097183e+08  5.571181e+07  ...   41651.05  5.510192e+09
     4    6.969695e+08  5.824780e+07  ...   26078.15  6.416704e+09
     5    6.557835e+08  6.595059e+07  ...   44077.72  6.710725e+09
     6    7.066638e+08  6.500270e+07  ...   26138.37  7.250115e+09
     7    6.647595e+08  8.153295e+07  ...   21099.70  6.493913e+09
     8    7.236017e+08  7.816405e+07  ...   27720.09  6.692602e+09
     9    4.854887e+08  8.381054e+07  ...   30999.67  5.299714e+09
     10   4.336106e+08  8.038748e+07  ...   39912.95  4.644334e+09
     11   2.701445e+08  9.301040e+07  ...   36643.70  3.553499e+09
Suma      5.843053e+09  7.847339e+08  ...  369609.19  5.943092e+10

[12 rows x 22 columns]

In [14]:
# Borrar las filas con NaN de la siguiente manera es análoga a la anterior:

# info_top_10.dropna(axis = 'index', subset = ['Pnet(kg)'], how = 'any', inplace = True) 
# the parameter 'axis = index' and 'how = any' are the default
# info_top_10

In [15]:
# Lo siguiente es análogo a la "tabla dinámica" anterior sin los totales ('margins')
# Con un formato tabular (tabla común). Seguro te copa más este

info_top_10 = info_top_10.groupby(['Año', 'Mes', 'NCM'], as_index = False).agg({'Pnet(kg)':'sum', 'FOB(u$s)':'sum'})
info_top_10['USD/KG'] =  info_top_10['FOB(u$s)'] / info_top_10['Pnet(kg)']

# Lo que busco es el ratio usd/kg, para ver cuál es el producto más caro de este top 10
# Para visualizar todo, uso el comando 'pd.set_option()'

pd.set_option('display.max_rows', info_top_10.shape[0]+1) # me aseguro de ver todas las filas del dataset
info_top_10

,Año,Mes,NCM,Pnet(kg),FOB(u$s),USD/KG
0,2020,1,10059010,1.862206e+09,3.143997e+08,0.168832
1,2020,1,12024200,4.957985e+07,6.060587e+07,1.222389
2,2020,1,12060090,1.667679e+06,1.245615e+06,0.746915
3,2020,1,22042100,1.181092e+07,4.306533e+07,3.646230
4,2020,1,23040010,1.654599e+09,5.323981e+08,0.321769
5,2020,1,30049099,2.027056e+05,3.377585e+06,16.662519
6,2020,1,38220090,3.682366e+04,1.252932e+06,34.025195
7,2020,1,39269090,1.749466e+05,1.139047e+06,6.510826
8,2020,1,73269090,5.564177e+05,2.014943e+06,3.621277
9,2020,1,99980500,4.390042e+04,1.567760e+05,3.571173


In [63]:
pd.reset_option('max_rows') # reseteo el máximo de filas que veo

print("La media USD/KG del top 10 más exportados es", info_top_10['USD/KG'].mean(), '\n') 
# I will look the NCMs whose USD/KG ratio is bigger than the total average

grouped = info_top_10[info_top_10['USD/KG'] > info_top_10['USD/KG'].mean()]
grouped.sort_values(by= ['USD/KG'], ascending = False, inplace = True)
grouped

La media USD/KG del top 10 más exportados es 7.381922868044652 



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,Año,Mes,NCM,Pnet(kg),FOB(u$s),USD/KG
16,2020,2,38220090,22504.94,972809.47,43.226486
26,2020,3,38220090,24245.09,996357.37,41.095223
86,2020,9,38220090,22388.12,901258.56,40.256107
56,2020,6,38220090,30824.67,1129194.80,36.632827
66,2020,7,38220090,13850.71,478832.63,34.570981
6,2020,1,38220090,36823.66,1252932.20,34.025195
105,2020,11,30049099,161124.08,5339677.36,33.140157
46,2020,5,38220090,29418.25,916378.36,31.149996
96,2020,10,38220090,30238.36,916720.00,30.316459
75,2020,8,30049099,285720.59,8600237.91,30.100169


In [64]:
print(f'Los bienes más caros (por arriba de la media) del top 10 corresponden a {list(grouped["NCM"].unique())}')
print('Y son', grouped['USD/KG'].count(), 'transacciones en total\n')

grouped.reset_index(drop = True, inplace = True) # Reseteo el índice
grouped

Los bienes más caros (por arriba de la media) del top 10 corresponden a [38220090, 30049099, 39269090]
Y son 30 transacciones en total



,Año,Mes,NCM,Pnet(kg),FOB(u$s),USD/KG
0,2020,2,38220090,22504.94,972809.47,43.226486
1,2020,3,38220090,24245.09,996357.37,41.095223
2,2020,9,38220090,22388.12,901258.56,40.256107
3,2020,6,38220090,30824.67,1129194.80,36.632827
4,2020,7,38220090,13850.71,478832.63,34.570981
5,2020,1,38220090,36823.66,1252932.20,34.025195
6,2020,11,30049099,161124.08,5339677.36,33.140157
7,2020,5,38220090,29418.25,916378.36,31.149996
8,2020,10,38220090,30238.36,916720.00,30.316459
9,2020,8,30049099,285720.59,8600237.91,30.100169


In [183]:
# Los más caros del Top 10 en cada mes

table_2 = pd.pivot_table(info_top_10, values = ['USD/KG'], index = ['Año','Mes'],
                         aggfunc = {'USD/KG': max}).reset_index()

table_2 = pd.merge(table_2, info_top_10, how='inner', on='USD/KG', indicator=False)
table_2.drop(['Año_y','Mes_y'], axis=1)

,Año_x,Mes_x,USD/KG,NCM,Pnet(kg),FOB(u$s)
0,2020,1,34.025195,38220090,36823.66,1252932.20
1,2020,2,43.226486,38220090,22504.94,972809.47
2,2020,3,41.095223,38220090,24245.09,996357.37
3,2020,4,29.416468,38220090,33521.96,986097.66
4,2020,5,31.149996,38220090,29418.25,916378.36
5,2020,6,36.632827,38220090,30824.67,1129194.80
6,2020,7,34.570981,38220090,13850.71,478832.63
7,2020,8,30.100169,30049099,285720.59,8600237.91
8,2020,9,40.256107,38220090,22388.12,901258.56
9,2020,10,30.316459,38220090,30238.36,916720.00


In [17]:
# Una manera para borrar los datos confidenciales, también se puede tratar como string toda la serie

def corrige(x):
  for i in x:
    if i == ' s':
      return int(x.replace('s',''))

In [18]:
corrige(df['Pnet(kg)'])
df.head(10)

,Año,Mes,NCM,Pdes,Pnet(kg),FOB(u$s)
0,2020,1,1012100,208,NaN,s2 ...
1,2020,1,1012100,212,NaN,s2 ...
2,2020,1,1012100,225,NaN,s2 ...
3,2020,1,1012100,999,3250.0,35580
4,2020,1,1012900,208,NaN,s2 ...
5,2020,1,1012900,212,52450.0,1064400
6,2020,1,1012900,223,NaN,s3 ...
7,2020,1,1012900,225,NaN,s2 ...
8,2020,1,1012900,331,NaN,s4 ...
9,2020,1,1012900,335,NaN,s4 ...


In [19]:
# Ahora borro las filas con "NaN" (equivalente al 'Null' de SQL)

sin_confidenciales = df.dropna(axis = 0, subset = ['Pnet(kg)'], how='any') # "axis = 0" marca que trabaje sobre columnas
sin_confidenciales.sort_values(by = ['Año', 'Mes', 'NCM', 'Pdes'], ascending = True, inplace = True)
# sin_confidenciales.groupby(['Año', 'Mes', 'NCM', 'Pdes']).agg({'Pnet(kg)':'sum', 'FOB(u$s)':'sum'})
sin_confidenciales

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,Año,Mes,NCM,Pdes,Pnet(kg),FOB(u$s)
3,2020,1,1012100,999,3.250000e+03,35580
5,2020,1,1012900,212,5.245000e+04,1064400
14,2020,1,1012900,999,4.416000e+04,621350
42,2020,1,2012090,341,2.511300e+03,29316.6
44,2020,1,2012090,999,9.296800e+02,10154.3
...,...,...,...,...,...,...
143706,2020,11,99980500,426,2.080000e+02,1102.4
143709,2020,11,99980500,438,7.806000e+01,2182.12
143710,2020,11,99980500,451,5.133400e+02,1134.25
143712,2020,11,99980500,999,9.604210e+03,32634.7


In [20]:
# Guardo este dataset y también la tabla anterior, juntos, en un nuevo Excel con "ExcelWriter"
# En donde dice '(...)' pueden poner la ruta de su drive que deseen (con la extensión ".xlsx" al final)...
# y al ejectutar la celda se creará un Excel en la misma con los dos dataframes como hojas

writer = pd.ExcelWriter('/content/drive/MyDrive/(...).xlsx')
sin_confidenciales.to_excel(writer, sheet_name = 'Sin confidenciales', index = False, freeze_panes = (1,2))
table.to_excel(writer, sheet_name = 'Top 10 NCMs ', index_label = True, freeze_panes = (3,2))
writer.save()